### connect to wrds

In [1]:
import pandas as pd
# pd.get_option("display.max_columns")
pd.set_option("display.max_columns", None)

In [2]:
import wrds

In [3]:
conn = wrds.Connection()

Enter your WRDS username [n2ngo]: tiwata
Enter your password: ·············


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
Loading library list...
Done


### index constituents :SP Global 1200

In [8]:
list_index = conn.get_table(library='comp_global', table='g_idx_index')
list_index.to_csv("list_index.csv",index=False)

In [49]:
# S&P Global 1200 Index
sp1200_hist = conn.raw_sql("select * from comp_global.g_idxcst_his where gvkeyx = '150918'")
sp1200_hist.to_csv("spglobal1200_hist.csv",index=False)

In [33]:
sp1200_hist.head()

,gvkey,iid,gvkeyx,from,thru
0,001932,01W,150918,1989-09-11,None
1,001945,01W,150918,1989-09-11,2006-09-04
2,002000,01W,150918,1989-09-11,1999-04-16
3,002338,01W,150918,1989-09-11,1998-04-20
4,002338,01W,150918,2005-06-30,2016-06-20


In [15]:
from tqdm import tqdm

In [69]:
ids = pd.DataFrame()
for gvkey in tqdm(sp1200_hist.gvkey):
    tmp = conn.raw_sql("select * from comp_global.g_security where gvkey = '"+str(gvkey)+"'")
    ids = pd.concat([ids,tmp],axis=0)

100%|██████████| 1480/1480 [03:36<00:00,  6.84it/s]


In [71]:
ids.to_csv("spglobal1200_ids.csv",index=False)

### reprisk RRI

In [4]:
ids = pd.read_csv("spglobal1200_ids.csv")

In [31]:
ids.head()

,tic,gvkey,iid,cusip,dlrsni,dsci,epf,exchg,excntry,ibtic,isin,secstat,sedol,tpci,dldtei
0,NaN,1932,01W,NaN,NaN,ORD GBP.25,Y,194.0,GBR,@BAT,GB0002875804,A,0287580,0,NaN
1,NaN,1932,02W,NaN,NaN,ORD GBP.25 (SOUTH AFRICA LIS,N,177.0,ZAF,@41T,NaN,A,B3DCKS1,0,NaN
2,NaN,1932,03W,NaN,NaN,ORD GBP.25(GERMANY LIST),N,154.0,DEU,@B8T,NaN,A,5617278,0,NaN
3,NaN,1945,01W,NaN,1.0,ORD GBP.25,Y,194.0,GBR,@BOC,GB0001081206,I,0108120,0,2006-09-05
4,NaN,1945,02W,NaN,20.0,CL 2 PFD GBP1 CUM,N,194.0,GBR,NaN,GB0001081867,I,0108186,1,1998-01-02


In [10]:
import numpy as np

In [17]:
repriskids = pd.DataFrame()
for isin in tqdm(ids['isin'].unique()):
    if type(isin)==str:
        tmp = conn.raw_sql("select * from reprisk.v2_wrds_company_lookup where isin = '"+isin+"'")
        repriskids = pd.concat([repriskids,tmp],axis=0)

100%|██████████| 2208/2208 [05:16<00:00,  6.97it/s]


In [21]:
repriskids.to_csv("repriskids.csv",index=False)

In [22]:
repriskids.head()

,reprisk_id,company_name,isin
0,22,British American Tobacco PLC (BAT),GB0002875804
0,68203,BOC Group Ltd (British Oxygen Co),GB0001081206
0,8175,Rexam PLC,GB00BMHTPY25
0,26,BP PLC (British Petroleum),GB0007980591
0,26,BP PLC (British Petroleum),GB0001385250


In [27]:
repriskrri = pd.DataFrame()
for rrid in tqdm(repriskids['reprisk_id'].unique()):
    if type(rrid)==str:
        tmp = conn.raw_sql("select * from reprisk.pm_rri_data where reprisk_id = '"+rrid+"'")
        repriskrri = pd.concat([repriskrri,tmp],axis=0)

100%|██████████| 1165/1165 [03:31<00:00,  5.51it/s]


In [35]:
# repriskrri.shape
repriskrri.to_csv("reprisk_rri.csv",index=False)

(172872, 11)

In [36]:
reprisk = ids[['gvkey','isin']].merge(repriskids.merge(repriskrri,on='reprisk_id', how='right'),on='isin',how='right')
reprisk.drop('isin',axis=1).drop_duplicates().to_csv("reprisk.csv",index=False)

### financial data 1
-the firm’s total assets, 
-the firm’s ebitover-total-assets ratio,

In [81]:
conn.get_table(library='comp', table='g_funda',
               # columns = ['conm','isin','sedol','fyr',
               #                             'at','ebit'],
               obs=5)

,gvkey,indfmt,datafmt,consol,popsrc,acctstd,acqmeth,bspr,compst,curcd,final,fyear,fyr,ismod,pddur,scf,src,stalt,upd,datadate,fdate,pdate,accli,acco,aco,acofs,acox,acoxfs,acqdisn,acqdiso,act,adpac,am,amdc,ao,aoloch,aox,ap,apalch,apch,apdpfs,apfs,apo,apofs,aqc,artfs,asdis,asinv,at,atoch,autxr,bcef,bct,ca,capcst,capfl,capr1,capr2,capr3,caprt,caps,capx,capxfi,ceq,cfbd,cfere,cflaoth,cfo,cfpdo,cga,ch,che,cheb,chech,chee,chefs,chenfd,chfs,chs,cmp,cogs,crvnli,cshr,cstk,custadv,dbtb,dbte,dc,dcsfd,dcufd,dd1,dd1fs,dfpac,dfxa,dispoch,dlc,dlcch,dlcfs,dltis,dltr,dltt,do,doc,dp,dpact,dpc,dpdc,dpltb,dpsc,dpstb,dptb,dptc,dptic,dv,dvc,dvp,dvpdp,dvrec,dvrre,dvsco,dvt,ea,ebit,ebitda,eiea,eieac,emp,eqdivp,ero,exre,exres,exreu,fatb,fate,fatl,fatp,fca,fdfr,fea,fel,ffs,fiao,fincf,fininc,finle,finre,finvao,fopo,fsrco,fsrcopo,fsrcopt,fsrct,fuseo,fuset,gdwl,iaeq,iafxi,ialoi,ialti,iamli,iaoi,iapli,iarei,iassi,iasti,iati,ib,ibc,ibki,ibmii,icapt,idiis,idilb,idilc,idis,idist,idit,idits,iire,initb,intan,intand,intanp,intc,intfact,intfl,intiact,intoact,intpd,intpn,intrc,invch,invdsp,invfg,invo,invrm,invsvc,invt,invtfs,invwip,iobd,ioi,iore,ip,ipti,isgr,isgt,isgu,isoth,ist,ivaco,ivaeq,ivao,ivch,ivgod,ivi,ivncf,ivpt,ivst,ivstch,ivstfs,lcabg,lcacl,lcacr,lcag,lcal,lcalt,lcam,lcao,lcast,lcat,lco,lcofs,lcox,lct,lctfs,lcuacu,liqresn,liqreso,lndep,lninc,lnmd,lnrep,lo,lse,lt,ltdch,ltdlch,ltlo,mib,mibn,mibt,mic,mii,miseq,mtl,ncfliq,neqmi,nio,nit,noasub,nopi,np,npanl,npaore,nparl,npat,npfs,oancf,oancfc,oancfd,oiadp,oibdp,onbalb,onbale,opprft,pacqp,pcl,pi,pliach,ppegt,ppent,prc,prodv,prosai,prstkc,prv,psfix,pstk,pstkn,pstkr,ptran,purtshr,pvon,pvt,radp,ragr,rari,rati,rawmsm,rcl,re,recch,recco,reccofs,rect,rectfs,rectr,rectrfs,revt,ris,rlri,rlt,rpag,rv,rvbci,rvbpi,rvbti,rveqt,rvlrv,rvri,rvsi,rvti,rvupi,rvutx,saa,sal,sale,sbdc,sc,sco,seq,shrcap,siv,spi,sppch,sppiv,ssnp,sstk,stbo,stfixa,stinv,stio,stkch,subdis,subpur,tdsg,tdst,teq,transa,tsca,tstk,tstlta,tx,txc,txdb,txdc,txdi,txditc,txo,txop,txp,txpd,txpfs,txt,txw,ui,unl,unnp,vpac,vpo,wcap,wcapch,wcapchc,wcapopc,wcaps,wcapsa,wcapsu,wcapt,wcapu,xacc,xaccfs,xago,xagt,xcom,xcomi,xdvre,xeqo,xi,xido,xidoc,xindb,xindc,xins,xinst,xint,xintd,xivi,xivre,xlr,xnitb,xobd,xoi,xopr,xopro,xore,xpp,xppfs,xpr,xrd,xrent,xs,xsga,xstf,xstfo,xstfws,xt,iid,exchg,isin,sedol,ajexi,curcdi,cshoi,cshpria,epsexcon,epsexnc,epsincon,epsinnc,icapi,nicon,ninc,pv,tstkni,conm,costat,fic,loc,naicsh,sich,rank,au,auop
0,001166,INDL,HIST_STD,C,I,DS,None,GO,None,NLG,Y,1996.0,12.0,1.0,12.0,10.0,23.0,None,3.0,1996-12-31,None,None,None,None,17.783,None,17.783,None,None,None,395.228,None,0.000,None,0.0,9.245,0.0,70.089,None,-19.273,None,None,0.0,None,0.000,None,None,None,611.980,0.0,0.0,None,None,None,None,None,None,None,None,None,164.852,81.439,None,178.075,None,None,0.0,None,None,None,57.358,70.609,58.801,-1.443,57.358,None,None,None,None,None,332.410,None,None,0.331,-12.875,None,None,0.0,None,None,22.870,None,None,30.834,None,88.641,1.728,None,None,None,87.975,0.0,None,30.834,166.431,30.834,None,None,None,None,None,None,None,10.947,None,None,None,None,None,None,None,0.0,101.662,132.496,0.0,0.000,4.140,None,-39.315,8.800,None,None,None,None,None,None,0.0,None,None,None,None,0.000,20.029,None,None,None,None,-0.074,None,None,None,None,None,None,0.000,None,None,None,None,None,None,None,None,None,None,None,68.402,68.402,None,None,363.850,None,None,None,None,None,None,None,None,None,0.000,None,0.0,0.0,0.0,None,0.0,0.0,None,NaN,None,-28.221,None,16.611,-21.762,68.091,None,140.370,None,77.430,None,None,None,None,None,None,None,None,None,None,0.0,0.0,0.0,0.000,None,None,-80.649,None,13.251,-2.014,None,None,None,None,None,None,None,None,None,None,None,83.888,None,8.080,242.618,None,None,None,None,None,None,None,None,0.0,611.980,336.105,3.949,None,None,97.800,None,97.800,19.114,19.114,0.0,None,None,None,0.00,0.00,None,0.0,65.771,None,None,None,None,None,50.377,None,None,101.662,132.496,None,None,None,0.0,None,91.994,0.0,383.183,216.752,0.0,0.0,None,0.0,0.000,2.804,0.0,0.0,0.0

In [88]:
fdat1 = pd.DataFrame()
for isin,sedol in tqdm(zip(ids['isin'],ids['sedol']),total=ids.shape[0]):
# for isin,sedol in tqdm(zip(ids['isin'][:5],ids['isin'][:5]),total=5):
    if isin != "" and sedol != "":
        tmp = conn.raw_sql("select conm,isin,sedol,fyear,fyr,at,ebit from comp.g_funda where isin = '"+str(isin)+"' or sedol = '" + str(sedol) +"'"  )
        fdat1 = pd.concat([fdat1,tmp],axis=0)
fdat1.to_csv("financial_data1.csv",index=False)

100%|██████████| 3640/3640 [09:01<00:00,  6.72it/s]


### financial data 2
-the price-to-book ratio,   
-the firm’s total assets,   
-the firm’s ebitover-total-assets ratio,  
-the firm’s size  

In [94]:
fdat2 = pd.DataFrame()
for gvkey in tqdm(sp1200_hist.gvkey[:5]):
    tmp = conn.raw_sql("select gvkey,permno,cusip,ticker,adate,qdate,public_date,bm,mktcap from wrdsapps_finratio.firm_ratio where gvkey = '"+str(gvkey)+"'")
    fdat2 = pd.concat([fdat2,tmp],axis=0)
fdat2.to_csv("financial_data2.csv",index=False)

100%|██████████| 5/5 [00:00<00:00,  6.95it/s]


### return

In [40]:
# get corresponding permmno. find table for it

# get all return for the permnos

In [126]:
conn.raw_sql("select * from crsp_a_stock.msf limit 10")

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,vol,ret,bid,ask,shrout,cfacpr,cfacshr,altprc,spread,altprcdt,retx
0,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1985-12-31,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,-2.562500,NaN,1986-01-07,NaN
1,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-01-31,-2.50000,-4.437500,-4.375000,1771.0,NaN,None,None,3680.0,1.0,1.0,-4.375000,0.25000,1986-01-31,NaN
2,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-02-28,-3.25000,-4.375000,-3.250000,828.0,-0.257143,None,None,3680.0,1.0,1.0,-3.250000,0.25000,1986-02-28,-0.257143
3,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-03-31,-3.25000,-4.437500,-4.437500,1078.0,0.365385,None,None,3680.0,1.0,1.0,-4.437500,0.12500,1986-03-31,0.365385
4,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-04-30,-4.00000,-4.312500,-4.000000,957.0,-0.098592,None,None,3793.0,1.0,1.0,-4.000000,0.25000,1986-04-30,-0.098592
5,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-05-30,-3.06250,-4.218750,-3.109375,1074.0,-0.222656,None,None,3793.0,1.0,1.0,-3.109375,0.09375,1986-05-30,-0.222656
6,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-06-30,-2.90625,-3.296875,-3.093750,1069.0,-0.005025,None,None,3793.0,1.0,1.0,-3.093750,0.06250,1986-06-30,-0.005025
7,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-07-31,-2.59375,-3.437500,-2.843750,1163.0,-0.080808,None,None,3793.0,1.0,1.0,-2.843750,0.06250,1986-07-31,-0.080808
8,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-08-29,-1.03125,-2.625000,-1.093750,3049.0,-0.615385,None,None,3793.0,1.0,1.0,-1.093750,0.06250,1986-08-29,-0.615385
9,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-09-30,-0.96875,-1.281250,-1.031250,3551.0,-0.057143,None,None,3793.0,1.0,1.0,-1.031250,0.06250,1986-09-30,-0.057143


### the firm’s beta

In [31]:
conn.list_tables(library='wrdsapps_backtest_plus')

['signals_raw_plus']

In [30]:
beta =  conn.get_table(library='wrdsapps_backtest_basic', table='mastertable', 
                       #          columns = ['gvkey','permno','cusip','ticker','adate','qdate','public_date',
                       #                     'mktcap','bm','roe','de_ratio','rd_sale','adv_sale'],
                       obs=5)
beta

,permno,date,invsigma2,ret,mktcap,retx,rf,dgtw_xret,b_mkt,b_smb,b_hml,b_umd,ffxret,gvkey,ffi48,ffi48_desc,sp500_weight,crsp_weight,wrds_lc_500,wrds_mc_500,wrds_sc_2000,wrds_micro,wrds_index
0,10000.0,1986-06-30,913.669487,-0.005025,11.734594,-0.005025,0.0052,None,None,None,None,None,None,013007,9.0,Hshld,None,0.000004,None,None,None,0.000300,0.000005
1,10000.0,1986-07-31,804.226220,-0.080808,10.786344,-0.080808,0.0052,None,None,None,None,None,None,013007,9.0,Hshld,None,0.000004,None,None,None,0.000300,0.000005
2,10000.0,1986-08-31,685.662587,-0.615385,4.148594,-0.615385,0.0046,None,None,None,None,None,None,013007,9.0,Hshld,None,0.000002,None,None,None,0.000116,0.000002
3,10000.0,1986-09-30,794.590129,-0.057143,3.911531,-0.057143,0.0045,None,None,None,None,None,None,013007,9.0,Hshld,None,0.000002,None,None,None,0.000114,0.000002
4,10000.0,1986-10-31,749.217156,-0.242424,3.002344,-0.242424,0.0046,None,None,None,None,None,None,013007,6.0,Toys,None,0.000001,None,None,None,0.000085,0.000001
